# Task 7 – Basic Sales Summary using SQLite in Google Colab
**Steps included in this notebook:**
1. Create a sample SQLite database (`sales_data.db`) with a sales table.
2. Insert sample sales records.
3. Run SQL queries inside Python to get total quantity and revenue per product.
4. Print the results.
5. Plot a revenue bar chart.
6. Optionally download the DB and chart.

**Note:** Colab storage is temporary — download your files before closing the session.

In [ ]:
import sqlite3
from datetime import date

# Create & connect to DB
conn = sqlite3.connect("sales_data.db")
cur = conn.cursor()

# Create table
cur.execute("""
CREATE TABLE IF NOT EXISTS sales (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    tx_date TEXT NOT NULL,
    product TEXT NOT NULL,
    quantity INTEGER NOT NULL,
    price REAL NOT NULL
);
""")

# Sample data
rows = [
    ("Laptop",   2, 75000.00),
    ("Laptop",   1, 78000.00),
    ("Mouse",    5,   600.00),
    ("Mouse",    3,   650.00),
    ("Keyboard", 2,  1500.00),
    ("Keyboard", 4,  1400.00),
    ("Monitor",  1, 12000.00),
    ("Monitor",  2, 11500.00),
    ("Headset",  3,  2200.00),
    ("Headset",  2,  2100.00),
]

# Insert data
cur.executemany(
    "INSERT INTO sales (tx_date, product, quantity, price) VALUES (?, ?, ?, ?)",
    [(str(date.today()), p, q, pr) for (p, q, pr) in rows]
)

conn.commit()
conn.close()
print("Database created and populated in Colab.")


In [ ]:
import pandas as pd

conn = sqlite3.connect("sales_data.db")

query = """
SELECT 
    product,
    SUM(quantity) AS total_qty,
    ROUND(SUM(quantity * price), 2) AS revenue
FROM sales
GROUP BY product
ORDER BY revenue DESC;
"""

df = pd.read_sql_query(query, conn)
conn.close()

print("=== Sales Summary by Product ===")
print(df)


In [ ]:
import matplotlib.pyplot as plt

plt.bar(df["product"], df["revenue"])
plt.xlabel("Product")
plt.ylabel("Revenue")
plt.title("Revenue by Product")
plt.tight_layout()
plt.show()


In [ ]:
from google.colab import files

# Save chart
plt.bar(df["product"], df["revenue"])
plt.xlabel("Product")
plt.ylabel("Revenue")
plt.title("Revenue by Product")
plt.tight_layout()
plt.savefig("sales_chart.png", dpi=150)

# Download the chart
files.download("sales_chart.png")

# Download the DB
files.download("sales_data.db")
